[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alexj-smolin/ds-school/blob/main/part4/HW4_1.ipynb)

In [1]:
import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# data_dir = '/content/drive/MyDrive/Sber_DS/NN/data/'
data_dir = './data/'
train_lang = 'en'

In [3]:
class DatasetSeq(Dataset):
    def __init__(self, data_dir, train_lang='en'):
        #open file
        with open(data_dir + train_lang + '.train', 'r') as f:
            train = f.read().split('\n\n')

        # delete extra tag markup
        train = [x for x in train if not '_ ' in x]
        #init vocabs of tokens for encoding {<str> token: <int> id}
        self.target_vocab = {} # {p: 1, a: 2, r: 3, pu: 4}
        self.word_vocab = {} # {cat: 1, sat: 2, on: 3, mat: 4, '.': 5}
        self.char_vocab = {} # {c: 1, a: 2, t: 3, ' ': 4, s: 5}

        # Cat sat on mat. -> [1, 2, 3, 4, 5]
        # p    a  r  p pu -> [1, 2, 3, 1, 4]
        # chars  -> [1, 2, 3, 4, 5, 2, 3, 4]

        #init encoded sequences lists (processed data)
        self.encoded_sequences = []
        self.encoded_targets = []
        self.encoded_char_sequences = []
        # n=1 because first value is padding
        n_word = 1
        n_target = 1
        n_char = 1
        for line in train:
            sequence = []
            target = []
            chars = []
            for item in line.split('\n'):
                if item != '':
                    word, label = item.split(' ')

                    if self.word_vocab.get(word) is None:
                        self.word_vocab[word] = n_word
                        n_word += 1
                    if self.target_vocab.get(label) is None:
                        self.target_vocab[label] = n_target
                        n_target += 1
                    for char in word:
                        if self.char_vocab.get(char) is None:
                            self.char_vocab[char] = n_char
                            n_char += 1
                    sequence.append(self.word_vocab[word])
                    target.append(self.target_vocab[label])
                    chars.append([self.char_vocab[char] for char in word])
            self.encoded_sequences.append(sequence)
            self.encoded_targets.append(target)
            self.encoded_char_sequences.append(chars)

    def __len__(self):
        return len(self.encoded_sequences)

    def __getitem__(self, index):
        return {
            'data': self.encoded_sequences[index], # [1, 2, 3, 4, 6] len=5
            'char': self.encoded_char_sequences[index],# [[1,2,3], [4,5], [1,2], [2,6,5,4], []] len=5
            'target': self.encoded_targets[index], #  (1)
        }

In [4]:
dataset = DatasetSeq(data_dir)

In [5]:
#padding
# seq1 = [1, 2, 3, 4]
# seq2 = [9, 7, 6, 4, 3, 7, 5]
# pad seq1 equal seq2
# seq1 = [1, 2, 3, 4, 0, 0, 0]
# concat(seq1, seq2) [[1, 2, 3, 4, 0, 0, 0],
#                     [9, 7, 6, 4, 3, 7, 5]]

In [6]:
def collate_fn(batch):
    data = []
    target = []
    for item in batch:
        data.append(torch.as_tensor(item['data']))
        target.append(torch.as_tensor(item['target']))
    # pad different length sequences
    data = pad_sequence(data, batch_first=True, padding_value=0)
    target = pad_sequence(target, batch_first=True, padding_value=0)

    return {'data': data, 'target': target}

In [7]:
class RNNCellPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        self.rnn_cell = nn.GRUCell(emb_dim, hidden_dim)
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)
        self.hidden_dim = hidden_dim

    def forward(self, x): # B x T
        b, t = x.size()
        emb = self.word_emb(x) # B x T x Ebm_dim
        rnn_out = []
        hidden = torch.zeros((b, self.hidden_dim), device=x.device)
        for i in range(t):
            hidden = self.gru_cell(emb[:, i, :], # emb[:, i, :]: B x Emb_dim
                                   hidden) # hidden: B x Hid_dim
            rnn_out.append(hidden.unsqueeze(1)) # B x 1 x Hid_dim
        rnn_out = torch.cat(rnn_out, dim=1) # B x T x Hid_dim

        return self.clf(self.do(rnn_out))


In [8]:
# Изменения:
# * заменил GRU на RNN с 3 скрытыми слоями + dropout
# * увеличил размер hidden_dim и размер батча
# * учил на устройстве Apple M1 (через бэкэнд торча MPS)
# * качество после 10 эпох получилось сравнимым с обучением GRU

class RNNPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # batch_first = False: T x B x Vec
        # batch_first = True: B x T x Vec
        self.rnn = nn.RNN(emb_dim, hidden_dim, num_layers=3, batch_first=True, dropout=0.1) 
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)
        self.hidden_dim = hidden_dim

    def forward(self, x):
        emb = self.word_emb(x)
        hidden, _ = self.rnn(emb)

        return self.clf(self.do(hidden))
        

In [9]:
#hyper params
vocab_size = len(dataset.word_vocab) + 1
n_classes = len(dataset.target_vocab) + 1
n_chars = len(dataset.char_vocab) + 1
#TODO try to use other model parameters
emb_dim = 256
hidden = 512
n_epochs = 10
batch_size = 128

# cuda_device = -1
# device = f'cuda:{cuda_device}' if cuda_device != -1 else 'cpu'
device = 'mps'
len(dataset) // batch_size, 'batches'

(165, 'batches')

In [10]:
model = RNNPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
model.train()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [11]:
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    run_loss = 0.
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1), 
                         )
        loss.backward()
        optim.step()
        run_loss += loss.item()
        if i % 50 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
    print(f'epoch: {epoch}, final running loss: {run_loss / len(dataloader)}')
    print('-' * 64)

    torch.save(model.state_dict(), f'./rnn_chkpt_{epoch}.pth')

epoch: 0, step: 0, loss: 2.9697835445404053
epoch: 0, step: 50, loss: 0.3920879364013672
epoch: 0, step: 100, loss: 0.1905272901058197
epoch: 0, step: 150, loss: 0.23137880861759186
epoch: 0, final running loss: 0.3313199050950282
----------------------------------------------------------------
epoch: 1, step: 0, loss: 0.1871958076953888
epoch: 1, step: 50, loss: 0.18598927557468414
epoch: 1, step: 100, loss: 0.17706027626991272
epoch: 1, step: 150, loss: 0.1457369178533554
epoch: 1, final running loss: 0.16112508593183575
----------------------------------------------------------------
epoch: 2, step: 0, loss: 0.10315850377082825
epoch: 2, step: 50, loss: 0.16173872351646423
epoch: 2, step: 100, loss: 0.09664632380008698
epoch: 2, step: 150, loss: 0.13409478962421417
epoch: 2, final running loss: 0.12256829488006506
----------------------------------------------------------------
epoch: 3, step: 0, loss: 0.09604844450950623
epoch: 3, step: 50, loss: 0.1097358837723732
epoch: 3, step: 

In [12]:
#example
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model.eval()
    predict = model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l-1] for l in labels])

['PRON', 'VERB', 'ADV', 'SCONJ', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'PRON']
